In [44]:
import json
import numpy as np
import os
import csv
from sklearn import svm

In [102]:
# read result from training and estimation
confi_file = "../../configurations/configurations_membership.csv"
res_path = "../../../../sPMM_data/Normal/submit_CHTC/membership_{}" # output from chtc is saved in res_path.format(set_type)
res_path = "../../../../sPMM_data/submit_CHTC/membership_{}" # output from chtc is saved in res_path.format(set_type)
confi = np.genfromtxt(confi_file, delimiter=',', dtype=float, skip_header=1)
cnt = 0
for set_type in ["balanced", "unbalanced"]:
    for i in range(1, confi.shape[0] + 1):
        confi_i = confi[i - 1] # corresponding configuration
        set_index = int(confi_i[4]) # set index
        k0_train, k1_train = int(confi_i[8]), int(confi_i[9])

        save_path = './{}_{}'.format(set_type, set_index)
        if not os.path.isdir(save_path): os.mkdir(save_path)


        # supervised
        if os.path.isfile(res_path.format(set_type) + "/supervised_{}.txt".format(i)):
            cnt += 1
            save_to = save_path + "/supervised_{}_{}".format(k0_train, k1_train)
            if not os.path.isdir(save_to): os.mkdir(save_to)

            with open(res_path.format(set_type) + "/supervised_{}.txt".format(i), 'r') as f:
                x = f.read()
            supervised_res = json.loads(x)  
            # training
            for para in ['rho', 'Mu', 'Lambda']:
                np.savetxt(save_to + "/{}.csv".format(para), np.array(supervised_res['estimation'][para]), delimiter=',', header='', fmt="%.5f")
            np.savetxt(save_to + "/G_training.csv", np.array(supervised_res['estimation']["G"]), delimiter=',', header='', fmt="%.5f")
            # prediction 
            for ke in ['supervised_prediction', "unsupervised_prediction"]:
                for para in supervised_res[ke].keys():
                    np.savetxt(save_to + "/{}.csv".format(para), np.array(supervised_res[ke][para]), delimiter=',', header='', fmt="%.5f")

        # unsupervised
        if os.path.isfile(res_path.format(set_type) + "/unsupervised_{}.txt".format(i)):
            cnt += 1
            save_to = save_path + "/unsupervised_{}_{}".format(k0_train, k1_train)
            if not os.path.isdir(save_to): os.mkdir(save_to)

            with open(res_path.format(set_type) + "/unsupervised_{}.txt".format(i), 'r') as f:
                x = f.read()
            unsupervised_res = json.loads(x)  
            # training
            for para in ['rho', 'Mu', 'Lambda']:
                np.savetxt(save_to + "/{}.csv".format(para), np.array(unsupervised_res['estimation'][para]), delimiter=',', header='', fmt="%.5f")
            np.savetxt(save_to + "/U_training.csv", np.array(unsupervised_res['estimation']["U"]), delimiter=',', header='', fmt="%.5f")
            # prediction 
            for para in ["U", "Y"]:
                np.savetxt(save_to + "/{}.csv".format(para), np.array(unsupervised_res["prediction"][para]), delimiter=',', header='', fmt="%.5f")

        
    

In [58]:
# read results from estimating with true topics
confi_dataset_file = "../../configurations/configurations_membership_dataset.csv"
res_path = "../../../../sPMM_data/submit_CHTC/membership_{}/learn_with_truth"

confi = np.genfromtxt(confi_dataset_file, delimiter=',', dtype=float, skip_header=1)

missing = []

for set_type in ["balanced", "unbalanced"]:
    for set_index in range(1, confi.shape[0] + 1): 
        save_path = './{}_{}/true'.format(set_type, set_index)
        if not os.path.isdir(save_path): os.mkdir(save_path)
        for para in ["U", "G", "Y"]:
            fm = res_path.format(set_type) + "/{}_{}.csv".format(para, set_index)
            if os.path.isfile(fm):
                with open(fm, "r") as f:
                    x = f.read()
                with open(save_path + "/{}_{}.csv".format(para, set_index), "w") as f:
                    f.write(x)
            else:
                missing.append("/{}_{}.csv".format(para, set_index))
missing
# ['/U_25.csv', '/U_26.csv', '/U_27.csv', '/U_52.csv', '/U_53.csv', '/U_54.csv', '/U_72.csv', '/U_79.csv', '/U_80.csv', '/U_81.csv', '/U_25.csv', '/U_26.csv', '/U_27.csv', '/U_52.csv', '/U_53.csv', '/U_54.csv', '/U_79.csv', '/U_80.csv', '/U_81.csv']



In [54]:
# generate prediction for learn_with_truth
confi_dataset_file = "../../configurations/configurations_membership_dataset.csv"
confi = np.genfromtxt(confi_dataset_file, delimiter=',', dtype=float, skip_header=1)

for set_type in ["balanced", "unbalanced"]:
    for set_index in range(1, confi.shape[0] + 1): 
    # for set_index in range(1, 2): 
        res_path = './{}_{}/true/'.format(set_type, set_index)
        Ntrain, Ntest = confi[set_index - 1, -2: ].astype(int)
        if not os.path.isfile(res_path + "U_{}.csv".format(set_index)):
            continue
        U = np.genfromtxt(res_path + "U_{}.csv".format(set_index), delimiter = ',')
        
        Y_train = np.genfromtxt('./{}_{}/data/training_set/Y.csv'.format(set_type, set_index), delimiter = ',')

        classifier = svm.SVC(C = 2,kernel = 'rbf', decision_function_shape = 'ovo') 
        classifier.fit(U[: Ntrain, ], Y_train)
        Y_pre = classifier.predict(U[Ntrain: ,])
        np.savetxt(res_path + "Y_un.csv", Y_pre, delimiter = ',', header = '')
            # save_to + "/{}.csv".format(para), np.array(unsupervised_res['estimation'][para]), delimiter=',', header='', fmt="%.5f")
            
           



In [27]:
# read real data to the same set
confi_dataset_file = "../../configurations/configurations_membership_dataset.csv"
data_path = {"balanced": "../../dataset/dataset_{}.txt", "unbalanced":  "../../dataset/unbalanced_{}.txt"}

confi = np.genfromtxt(confi_dataset_file, delimiter=',', dtype=float, skip_header=1)

for set_type in ["balanced", "unbalanced"]:
    for i in range(1, confi.shape[0] + 1):
        # read data
        set_index = i
        with open(data_path[set_type].format(i), "r") as f:
            x = f.read()
        dat = json.loads(x)

        save_path = './{}_{}/data'.format(set_type, set_index)
        if not os.path.isdir(save_path): os.mkdir(save_path)

        for sec in ["model", "training_set", "test_set"]:
            save_path_det = model_path = save_path + '/' + sec
            if not os.path.isdir(save_path_det): os.mkdir(save_path_det) 
            for para in dat[sec].keys():
                if para != 'a':
                    # print(save_path_det + '/{}.csv'.format(para))
                    np.savetxt(save_path_det + '/{}.csv'.format(para), np.array(dat[sec][para]), delimiter=',', header='', fmt="%.5f")
                
        